In [1]:
import itk

from itkwidgets import view

In [3]:
PixelType = itk.F
Dimension = 3
    
# Read tre file
tubesReader = itk.SpatialObjectReader[Dimension].New()
tubesReader.SetFileName("data/Normal071-VascularNetwork.tre")
tubesReader.Update()
tubes = tubesReader.GetGroup()

# Read mra image
mraReader = itk.ImageFileReader[itk.Image[PixelType, Dimension]].New()
mraReader.SetFileName("data/Normal071-MRA.mha")
mraReader.Update()
mra = mraReader.GetOutput()

In [5]:
view(image=mra, point_sets=tubes)

Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[0.8392157, 0.       , 0.       ]], dtype…

In [6]:
print("Number of objects = ", tubes.GetNumberOfChildren())

sobj = tubes.GetChildren(0, "VesselTube")
baseSO = sobj[0]

print("Children are all maintained as base spatial object types - not as tubes, groups, landmarks, etc.")
print("  You can print info about the object:")
print(baseSO)
print("  But you cannot call member functions in derived classes (e.g., TubeSO member functions):")
print(baseSO.GetNumberOfPoints())   # This command fails - it should fail...and the next cell shows the fix...

Number of objects =  265
Children are all maintained as base spatial object types - not as tubes, groups, landmarks, etc.
  You can print info about the object:
TubeSpatialObject (000002AC1F6EA100)
  TubeSpatialObject(000002AC1F6EA100)
  End Type : 1
  Parent Point : -1
  Root : 1
  PointBasedSpatialObject(000002AC1F6EA100)
  Number of points: 4330
  RTTI typeinfo:   class itk::TubeSpatialObject<3,class itk::TubeSpatialObjectPoint<3> >
  Reference Count: 5
  Modified Time: 4480
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 0
  UpdateMTime: 0
  RealTimeStamp: 0 seconds 
  Id:0
  TypeName:VesselTubeSpatialObject
  ParentId:265
  Parent:000002AC23E9E5A0
  LargestPossibleRegion:ImageRegion (000002AC1F6EA240)
  Dimension: 3
  Index: [0, 0, 0]
  Size: [0, 0, 0]

  RequestedRegion:ImageRegion (000002AC1F6EA278)
  Dimension: 3
  Index: [0, 0, 0]
  Size: [0,

AttributeError: 'itkSpatialObject3' object has no attribute 'GetNumberOfPoints'

In [7]:
print("One option is to downcast each object in the children list, as needed")

import itkExtras

baseSOTyped = itkExtras.down_cast(baseSO)

print("  Now we can call member functions of the derived class")
print(baseSOTyped.GetNumberOfPoints())   # This command succeeds on the down-casted variable

One option is to downcast each object in the children list, as needed
  Now we can call member functions of the derived class
4330


In [8]:
print("Another option is to use CastSpatialObject class to convert base spatial objects of a particular type...")
castSO = itk.CastSpatialObjectFilter[3].New()
castSO.SetInput(tubes)
tubesSO = castSO.GetTubes()

Another option is to use CastSpatialObject class to convert base spatial objects of a particular type...


In [9]:
print("Then you can access its member functions...")
tube = tubesSO[0]
tube.GetPoints()[1].GetPositionInObjectSpace()

Then you can access its member functions...


itkPointD3 ([121.292, 94.4135, 0.313261])

In [19]:
p = list(map(lambda x: tube.GetPoint(x).GetPositionInObjectSpace(), range(tube.GetNumberOfPoints())))
i = list(map(lambda x: mra.TransformPhysicalPointToIndex( x ), p ))
for x in i:
    mra.SetPixel(x, 10000)
    
view(mra)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageF3; proxy o…